In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [5]:
data=pd.read_csv("Churn_Modelling.csv")

In [6]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [8]:
le=LabelEncoder()
data['Gender']=le.fit_transform(data['Gender'])

In [11]:
ohe=OneHotEncoder()
Geography_encoded=ohe.fit_transform(data[['Geography']])
Geography_encoded

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [17]:
print(Geography_encoded.toarray())

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [21]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [23]:
geodata=pd.DataFrame(Geography_encoded.toarray(), columns=ohe.get_feature_names_out(['Geography']))
geodata

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [37]:
data.drop(['Geography'],axis=1,inplace=True)

In [33]:
data=pd.concat([data,geodata],axis=1)

In [38]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [39]:
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [40]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Scale the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [41]:
## Save the encoders and scaler
import pickle
with open('label_encoder_gender_reg.pkl','wb') as file:
    pickle.dump(le,file)

with open('one_hot_encoder_geo_reg.pkl','wb') as file:
    pickle.dump(ohe,file)

with open('scaler.pkl_reg','wb') as file:
    pickle.dump(scaler,file)

# ANN Regression Problem Statement

In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [45]:
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression problem
])

In [46]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
## Set up early stopping
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [48]:
## Training the moddel

history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 5ms/step - loss: 100385.2500 - mae: 100385.2500 - val_loss: 98536.9062 - val_mae: 98536.9062
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 99672.8125 - mae: 99672.8125 - val_loss: 97083.1484 - val_mae: 97083.1484
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 97129.8672 - mae: 97129.8672 - val_loss: 93342.5547 - val_mae: 93342.5547
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 92098.4922 - mae: 92098.4922 - val_loss: 87090.1719 - val_mae: 87090.1719
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 84725.1406 - mae: 84725.1406 - val_loss: 78922.4531 - val_mae: 78922.4531
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 75933.3281 - mae: 75933.3281 - val_loss: 70164.0000 - val_mae: 70164.0000
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 67229.7734 - mae: 67229.773

In [49]:
#Evaluate the model on test data
test_loss, test_mae=model.evaluate(X_test,y_test)

print(f'Teat Mae: {test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50331.6484 - mae: 50331.6484
Teat Mae: 50331.6484375


In [50]:
model.save('regression_model.h5')

c:\Users\chara\anaconda3\envs\nlpenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
